<a href="https://colab.research.google.com/github/VasaviLagishetty/COVID19/blob/master/COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
from datetime import datetime

In [32]:
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])
df.head()

,attributes,geometry
0,"{'OBJECTID': 1, 'Province_State': 'Abruzzo', '...","{'x': 13.398438230000068, 'y': 42.35122196000003}"
1,"{'OBJECTID': 2, 'Province_State': 'Acre', 'Cou...","{'x': -70.81199999999995, 'y': -9.023799999999..."
2,"{'OBJECTID': 3, 'Province_State': 'Adygea Repu...","{'x': 40.152042100000074, 'y': 44.69390060000006}"
3,"{'OBJECTID': 4, 'Province_State': 'Aguascalien...","{'x': -102.29159999999996, 'y': 21.88530000000..."
4,"{'OBJECTID': 5, 'Province_State': 'Aichi', 'Co...","{'x': 137.21162100000004, 'y': 35.035551000000..."


In [3]:
df["attributes"][0]

{'Confirmed': 3257,
 'Country_Region': 'Italy',
 'Deaths': 415,
 'Last_Update': 1591407175000,
 'Lat': 42.35122196,
 'Long_': 13.39843823,
 'OBJECTID': 1,
 'Province_State': 'Abruzzo',
 'Recovered': 2172}

In [4]:
df['geometry'][0]

{'x': 13.398438230000068, 'y': 42.35122196000003}

In [5]:
data_list = df["attributes"].tolist()
data = pd.DataFrame(data_list)
data = data.set_index("OBJECTID")
data = data.reset_index(drop = True)
data

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,Abruzzo,Italy,1.591407e+12,42.351222,13.398438,3257,2172,415
1,Acre,Brazil,1.591407e+12,-9.023800,-70.812000,7021,0,181
2,Adygea Republic,Russia,1.591407e+12,44.693901,40.152042,962,519,9
3,Aguascalientes,Mexico,1.591407e+12,21.885300,-102.291600,1072,703,48
4,Aichi,Japan,1.591407e+12,35.035551,137.211621,509,462,34
...,...,...,...,...,...,...,...,...
681,Ohio,US,1.591407e+12,40.388783,-82.764915,37758,0,2357
682,Alabama,US,1.591407e+12,32.318200,-86.902300,19387,0,676
683,North Carolina,US,1.591407e+12,35.630066,-79.806419,33295,0,1015
684,District of Columbia,US,1.591407e+12,38.897438,-77.026817,9199,0,479


In [6]:
bool_series = pd.isnull(data['Last_Update']) 
data[bool_series]

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
640,American Samoa,US,NaN,-14.271,-170.132,0,0,0
651,United States Virgin Islands,US,NaN,18.350,-64.933,0,0,0


In [7]:
data = data[data['Last_Update'].notna()]
data["Province_State"].fillna(value="", inplace=True)

data

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,Abruzzo,Italy,1.591407e+12,42.351222,13.398438,3257,2172,415
1,Acre,Brazil,1.591407e+12,-9.023800,-70.812000,7021,0,181
2,Adygea Republic,Russia,1.591407e+12,44.693901,40.152042,962,519,9
3,Aguascalientes,Mexico,1.591407e+12,21.885300,-102.291600,1072,703,48
4,Aichi,Japan,1.591407e+12,35.035551,137.211621,509,462,34
...,...,...,...,...,...,...,...,...
681,Ohio,US,1.591407e+12,40.388783,-82.764915,37758,0,2357
682,Alabama,US,1.591407e+12,32.318200,-86.902300,19387,0,676
683,North Carolina,US,1.591407e+12,35.630066,-79.806419,33295,0,1015
684,District of Columbia,US,1.591407e+12,38.897438,-77.026817,9199,0,479


In [8]:
def convertTime(t):
  t = int(t)
  return datetime.fromtimestamp(t)

data['Last_Update'] = data['Last_Update']/1000
data['Last_Update'] = data['Last_Update'].apply(convertTime)
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths
0,Abruzzo,Italy,2020-06-06 01:32:55,42.351222,13.398438,3257,2172,415
1,Acre,Brazil,2020-06-06 01:32:55,-9.023800,-70.812000,7021,0,181
2,Adygea Republic,Russia,2020-06-06 01:32:55,44.693901,40.152042,962,519,9
3,Aguascalientes,Mexico,2020-06-06 01:32:55,21.885300,-102.291600,1072,703,48
4,Aichi,Japan,2020-06-06 01:32:55,35.035551,137.211621,509,462,34
...,...,...,...,...,...,...,...,...
681,Ohio,US,2020-06-06 01:32:55,40.388783,-82.764915,37758,0,2357
682,Alabama,US,2020-06-06 01:32:55,32.318200,-86.902300,19387,0,676
683,North Carolina,US,2020-06-06 01:32:55,35.630066,-79.806419,33295,0,1015
684,District of Columbia,US,2020-06-06 01:32:55,38.897438,-77.026817,9199,0,479


In [0]:
df_total = data.groupby('Country_Region',as_index=False).agg({'Confirmed' : 'sum','Recovered' : 'sum','Deaths' : 'sum'}) 

In [10]:
df_total

,Country_Region,Confirmed,Recovered,Deaths
0,Afghanistan,18969,1762,309
1,Albania,1212,910,33
2,Algeria,9935,6453,690
3,Andorra,852,741,51
4,Angola,86,21,4
...,...,...,...,...
183,West Bank and Gaza,464,377,3
184,Western Sahara,9,6,1
185,Yemen,469,23,111
186,Zambia,1089,912,7


In [11]:
total_confirmed = data['Confirmed'].sum()
total_recovered = data['Recovered'].sum()
total_deaths = data['Deaths'].sum()
print("Total confirmed cases till",data['Last_Update'][0],"are",total_confirmed)
print("Total recovered cases till",data['Last_Update'][0],"are",total_recovered)
print("Total death cases till",data['Last_Update'][0],"are",total_deaths)

Total confirmed cases till 2020-06-06 01:32:55 are 6731824
Total recovered cases till 2020-06-06 01:32:55 are 3000221
Total death cases till 2020-06-06 01:32:55 are 394787


In [0]:
df_top10 = df_total.nlargest(10, "Confirmed")
top10_countries_1 = df_top10["Country_Region"].tolist()
top10_confirmed = df_top10["Confirmed"].tolist()

df_top10 = df_total.nlargest(10, "Recovered")
top10_countries_2 = df_top10["Country_Region"].tolist()
top10_recovered = df_top10["Recovered"].tolist()

df_top10 = df_total.nlargest(10, "Deaths")
top10_countries_3 = df_top10["Country_Region"].tolist()
top10_deaths = df_top10["Deaths"].tolist()

# Subplots

In [0]:
fig = make_subplots(
    rows = 4, cols = 6,
    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)

In [14]:
message = data["Country_Region"] + " " + data["Province_State"] + "<br>"
message += "Confirmed: " + data["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + data["Deaths"].astype(str) + "<br>"
message += "Recovered: " + data["Recovered"].astype(str) + "<br>"
message += "Last updated: " + data["Last_Update"].astype(str)
data.loc[:,'text'] = message
data

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Recovered,Deaths,text
0,Abruzzo,Italy,2020-06-06 01:32:55,42.351222,13.398438,3257,2172,415,Italy Abruzzo<br>Confirmed: 3257<br>Deaths: 41...
1,Acre,Brazil,2020-06-06 01:32:55,-9.023800,-70.812000,7021,0,181,Brazil Acre<br>Confirmed: 7021<br>Deaths: 181<...
2,Adygea Republic,Russia,2020-06-06 01:32:55,44.693901,40.152042,962,519,9,Russia Adygea Republic<br>Confirmed: 962<br>De...
3,Aguascalientes,Mexico,2020-06-06 01:32:55,21.885300,-102.291600,1072,703,48,Mexico Aguascalientes<br>Confirmed: 1072<br>De...
4,Aichi,Japan,2020-06-06 01:32:55,35.035551,137.211621,509,462,34,Japan Aichi<br>Confirmed: 509<br>Deaths: 34<br...
...,...,...,...,...,...,...,...,...,...
681,Ohio,US,2020-06-06 01:32:55,40.388783,-82.764915,37758,0,2357,US Ohio<br>Confirmed: 37758<br>Deaths: 2357<br...
682,Alabama,US,2020-06-06 01:32:55,32.318200,-86.902300,19387,0,676,US Alabama<br>Confirmed: 19387<br>Deaths: 676<...
683,North Carolina,US,2020-06-06 01:32:55,35.630066,-79.806419,33295,0,1015,US North Carolina<br>Confirmed: 33295<br>Death...
684,District of Columbia,US,2020-06-06 01:32:55,38.897438,-77.026817,9199,0,479,US District of Columbia<br>Confirmed: 9199<br>...


In [15]:
fig.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = data["Long_"],
        lat = data["Lat"],
        hovertext = data["text"],
        showlegend=False,
        marker = dict(
            size = 5,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(50, 50, 50)'
            ),
            cmin = 0,
            color = data['Confirmed'],
            cmax = data['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),    
    row=1, col=1
)

fig.update_geos(
    projection_type="natural earth",
    landcolor="grey",
    oceancolor="LightBlue",
    showocean=True,
    lakecolor="LightBlue"
)



In [16]:
# Confirmed Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_confirmed,
        title="Confirmed Cases",
    ),
    row=1, col=4
)

# Recovered Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered Cases",
    ),
    row=1, col=5
)

# Death Cases
fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths Cases",
    ),
    row=1, col=6
)

In [17]:
fig.add_trace(
    go.Bar(
        x=top10_countries_1,
        y=top10_confirmed, 
        name= "Confirmed Cases",
        marker=dict(color="Yellow"), 
        showlegend=True,
    ),
    row=2, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_2,
        y=top10_recovered, 
        name= "Recovered Cases",
        marker=dict(color="Green"), 
        showlegend=True),
    row=3, col=4
)

fig.add_trace(
    go.Bar(
        x=top10_countries_3,
        y=top10_deaths, 
        name= "Deaths Cases",
        marker=dict(color="crimson"), 
        showlegend=True),
    row=4, col=4
)

In [18]:
fig.update_layout(
    template="plotly_dark",
    title = "Global COVID-19 Cases (Last Updated: " + str(data["Last_Update"][0]) + ")",
    showlegend=True,
    legend_orientation="h",
    legend=dict(x=0.65, y = -0.1),
    font=dict(family='Candara', size=12, color='#FFFFFF')
)

In [0]:
# Saving to a HTML file
fig.write_html('COVID19.html', auto_open=True)

In [0]:
url = "https://raw.githubusercontent.com/VasaviLagishetty/COVID19/master/covid_19_data.csv"
dataset = pd.read_csv(url)

In [21]:
dataset

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
22184,22185,05/06/2020,Wyoming,US,2020-05-07 02:32:28,631.0,7.0,0.0
22185,22186,05/06/2020,Xinjiang,Mainland China,2020-05-07 02:32:28,76.0,3.0,73.0
22186,22187,05/06/2020,Yukon,Canada,2020-05-07 02:32:28,11.0,0.0,0.0
22187,22188,05/06/2020,Yunnan,Mainland China,2020-05-07 02:32:28,185.0,2.0,181.0


In [22]:
India_covid_data = dataset[dataset['Country/Region'] == "India"]
India_covid_data

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
430,431,01/30/2020,NaN,India,1/30/20 16:00,1.0,0.0,0.0
491,492,01/31/2020,NaN,India,1/31/2020 23:59,1.0,0.0,0.0
547,548,02/01/2020,NaN,India,1/31/2020 8:15,1.0,0.0,0.0
607,608,02/02/2020,NaN,India,2020-02-02T06:03:08,2.0,0.0,0.0
672,673,02/03/2020,NaN,India,2020-02-03T21:43:02,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...
20649,20650,05/02/2020,NaN,India,2020-05-03 02:32:28,39699.0,1323.0,10819.0
20972,20973,05/03/2020,NaN,India,2020-05-04 02:32:28,42505.0,1391.0,11775.0
21295,21296,05/04/2020,NaN,India,2020-05-05 02:32:34,46437.0,1566.0,12847.0
21618,21619,05/05/2020,NaN,India,2020-05-06 02:32:31,49400.0,1693.0,14142.0


In [23]:
India_covid_data = India_covid_data.drop(['SNo','Province/State',],axis=1)
India_covid_data

,ObservationDate,Country/Region,Last Update,Confirmed,Deaths,Recovered
430,01/30/2020,India,1/30/20 16:00,1.0,0.0,0.0
491,01/31/2020,India,1/31/2020 23:59,1.0,0.0,0.0
547,02/01/2020,India,1/31/2020 8:15,1.0,0.0,0.0
607,02/02/2020,India,2020-02-02T06:03:08,2.0,0.0,0.0
672,02/03/2020,India,2020-02-03T21:43:02,3.0,0.0,0.0
...,...,...,...,...,...,...
20649,05/02/2020,India,2020-05-03 02:32:28,39699.0,1323.0,10819.0
20972,05/03/2020,India,2020-05-04 02:32:28,42505.0,1391.0,11775.0
21295,05/04/2020,India,2020-05-05 02:32:34,46437.0,1566.0,12847.0
21618,05/05/2020,India,2020-05-06 02:32:31,49400.0,1693.0,14142.0


In [24]:
India_covid_data.dtypes

ObservationDate     object
Country/Region      object
Last Update         object
Confirmed          float64
Deaths             float64
Recovered          float64
dtype: object

In [25]:
cols = India_covid_data.columns[India_covid_data.dtypes.eq('float64')]
India_covid_data[cols] = India_covid_data[cols].astype(int)
India_covid_data.rename(columns={'Country/Region' : 'Country','ObservationDate':'Date'},inplace=True)
India_covid_data

,Date,Country,Last Update,Confirmed,Deaths,Recovered
430,01/30/2020,India,1/30/20 16:00,1,0,0
491,01/31/2020,India,1/31/2020 23:59,1,0,0
547,02/01/2020,India,1/31/2020 8:15,1,0,0
607,02/02/2020,India,2020-02-02T06:03:08,2,0,0
672,02/03/2020,India,2020-02-03T21:43:02,3,0,0
...,...,...,...,...,...,...
20649,05/02/2020,India,2020-05-03 02:32:28,39699,1323,10819
20972,05/03/2020,India,2020-05-04 02:32:28,42505,1391,11775
21295,05/04/2020,India,2020-05-05 02:32:34,46437,1566,12847
21618,05/05/2020,India,2020-05-06 02:32:31,49400,1693,14142


In [26]:
India_covid_data['Date'] = pd.to_datetime(India_covid_data['Date'])
India_covid_data['Date'] = India_covid_data['Date'].dt.strftime('%Y-%m-%d')
India_covid_data

,Date,Country,Last Update,Confirmed,Deaths,Recovered
430,2020-01-30,India,1/30/20 16:00,1,0,0
491,2020-01-31,India,1/31/2020 23:59,1,0,0
547,2020-02-01,India,1/31/2020 8:15,1,0,0
607,2020-02-02,India,2020-02-02T06:03:08,2,0,0
672,2020-02-03,India,2020-02-03T21:43:02,3,0,0
...,...,...,...,...,...,...
20649,2020-05-02,India,2020-05-03 02:32:28,39699,1323,10819
20972,2020-05-03,India,2020-05-04 02:32:28,42505,1391,11775
21295,2020-05-04,India,2020-05-05 02:32:34,46437,1566,12847
21618,2020-05-05,India,2020-05-06 02:32:31,49400,1693,14142


In [27]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=India_covid_data['Date'], y=India_covid_data['Confirmed'],
                    mode='lines',
                    name='confirmed'))

fig.add_trace(go.Scatter(x=India_covid_data['Date'], y=India_covid_data['Deaths'],
                    mode='lines',
                    name='deaths'))

fig.add_trace(go.Scatter(x=India_covid_data['Date'], y=India_covid_data['Recovered'],
                    mode='lines',
                    name='recovered'))

fig.update_layout(
    template="plotly_dark",
    title="INDIA COVID19 CASES",
    xaxis_title="Date",
    yaxis_title="No of cases",
    font=dict(
        family='Candara',
        size=13,
        color="#7f7f7f"
    )
)

fig.show()

In [28]:
India_covid_data

,Date,Country,Last Update,Confirmed,Deaths,Recovered
430,2020-01-30,India,1/30/20 16:00,1,0,0
491,2020-01-31,India,1/31/2020 23:59,1,0,0
547,2020-02-01,India,1/31/2020 8:15,1,0,0
607,2020-02-02,India,2020-02-02T06:03:08,2,0,0
672,2020-02-03,India,2020-02-03T21:43:02,3,0,0
...,...,...,...,...,...,...
20649,2020-05-02,India,2020-05-03 02:32:28,39699,1323,10819
20972,2020-05-03,India,2020-05-04 02:32:28,42505,1391,11775
21295,2020-05-04,India,2020-05-05 02:32:34,46437,1566,12847
21618,2020-05-05,India,2020-05-06 02:32:31,49400,1693,14142


In [0]:
import plotly.express as px
config = dict({'scrollZoom': False})

In [30]:
India_fig = px.choropleth(India_covid_data, locations="Country", 
                    locationmode='country names', color="Confirmed", 
                    hover_name="Country", range_color=[1,60000],
                    color_continuous_scale=["green","yellow","red"], 
                    animation_frame="Date",
                    title='India confirmed cases',scope='asia', height=750)
India_fig.show(config=config)

In [31]:
world = go.Figure()

world.add_trace(
    go.Scattergeo(
        locationmode = "country names",
        lon = data["Long_"],
        lat = data["Lat"],
        hovertext = data["text"],
        showlegend=False,
        marker = dict(
            size = 5,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = True,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(50, 50, 50)'
            ),
            cmin = 0,
            color = data['Confirmed'],
            cmax = data['Confirmed'].max(),
            colorbar_title="Confirmed Cases<br>Latest Update",  
            colorbar_x = -0.05
        )

    ),  
)

world.update_geos(
    projection_type="natural earth",
    landcolor="grey",
    oceancolor="LightBlue",
    showocean=True,
    lakecolor="LightBlue",
)

world.update_layout(
    template="plotly_dark",
    title_text = "Global COVID-19 Cases",
    title_x=0.5,
    showlegend=False,
    legend=dict(x=0.65, y = -0.1),
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#fff",        
    )
)

world.show()